# Prep 1: steps to install the packages

In [1]:
!pip install py_entitymatching==0.2.0
!pip install scipy
!pip install numpy
!pip install pandas

# [NEED MODIFICATION: Modify this cell to point to the file location]
# Prep 2: enter the file location on your harddisk

In [2]:
table_a = 'tableA.csv'
table_b = 'tableB.csv'
candidate_set = 'Task2_apply_rules_ds.csv'
prediction_set = 'Task2_cm_matching_al_ds.csv'

# Prep 3: reading the files into pandas dataframe

In [3]:
import pandas as pd
dfa = pd.read_csv(table_a)
dfb = pd.read_csv(table_b)
dfc = pd.read_csv(candidate_set)
dfp = pd.read_csv(prediction_set)

# Module: debug_blocker
# Description: debug the blocking rule using the below script to ensure you are not dropping true matches
# Note: You need to run Prep 1 and 2 in order to run this

In [6]:
# Example input format:
# Format of table_a:
# _id, attribute1, attribute2, ....., attributen

# Format of table_b:
# _id, attribute1, attribute2, ....., attributen

# Format of candidate_set
# A_id,B_id
# where A_id is _id from table_a and B_id is the _id column value from table_b

In [4]:
import py_entitymatching as em
import pandas as pd

def run_debug_blocker(table_a, table_b, table_a_key, table_b_key, candidate_set):
    dfl = em.read_csv_metadata(table_a, key=table_a_key)
    dfr = em.read_csv_metadata(table_b, key=table_b_key)

    # reading the candidate set and adding key
    dfcand = pd.read_csv(candidate_set)
    dfcand.drop_duplicates(inplace=True)
    dfcand.to_csv('cand_set_with_index.csv', index_label='id')

    dfcset = em.read_csv_metadata('cand_set_with_index.csv', key='id', ltable=dfl, 
                                  rtable=dfr, fk_ltable='A_id', fk_rtable='B_id')

    # running debug blocker to identify the records in A x B \ C
    debug_file = em.debug_blocker(dfcset, dfl, dfr)
    
    return debug_file

In [5]:
debug_file = run_debug_blocker(table_a, table_b, '_id', '_id', candidate_set)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [6]:
debug_file

,_id,similarity,ltable__id,rtable__id,ltable_Title,ltable_Date_of_Release,ltable_Description,ltable_Up_System,rtable_Title,rtable_Date_of_Release,rtable_Description,rtable_Up_System
0,0,0.955556,1673,1337,Law Abiding Citizen,2009,A frustrated man decides to take justice into his own hands after a plea bargain sets one of his...,CM,Law Abiding Citizen,"October 15, 2009",A frustrated man decides to take justice into his own hands after a plea bargain sets one of his...,CM
1,1,0.955556,1805,1337,Law Abiding Citizen,2009,A frustrated man decides to take justice into his own hands after a plea bargain sets one of his...,CM,Law Abiding Citizen,"October 15, 2009",A frustrated man decides to take justice into his own hands after a plea bargain sets one of his...,CM
2,2,0.955556,1761,1337,Law Abiding Citizen,2009,A frustrated man decides to take justice into his own hands after a plea bargain sets one of his...,CM,Law Abiding Citizen,"October 15, 2009",A frustrated man decides to take justice into his own hands after a plea bargain sets one of his...,CM
3,3,0.955556,1717,1337,Law Abiding Citizen,2009,A frustrated man decides to take justice into his own hands after a plea bargain sets one of his...,CM,Law Abiding Citizen,"October 15, 2009",A frustrated man decides to take justice into his own hands after a plea bargain sets one of his...,CM
4,4,0.955556,2313,4523,The Expendables 2,2012,"Mr. Church reunites the Expendables for what should be an easy paycheck, but when one of their m...",CM,The Expendables 2,"August 17, 2012","Mr. Church reunites the Expendables for what should be an easy paycheck, but when one of their m...",CM
5,5,0.955556,2264,4523,The Expendables 2,2012,"Mr. Church reunites the Expendables for what should be an easy paycheck, but when one of their m...",CM,The Expendables 2,"August 17, 2012","Mr. Church reunites the Expendables for what should be an easy paycheck, but when one of their m...",CM
6,6,0.955556,1873,5180,Hot Tub Time Machine,2010,A malfunctioning time machine at a ski resort takes a man back to 1986 with his two friends and ...,CM,Hot Tub Time Machine,"March 26, 2010",A malfunctioning time machine at a ski resort takes a man back to 1986 with his two friends and ...,CM
7,7,0.955556,2362,4523,The Expendables 2,2012,"Mr. Church reunites the Expendables for what should be an easy paycheck, but when one of their m...",CM,The Expendables 2,"August 17, 2012","Mr. Church reunites the Expendables for what should be an easy paycheck, but when one of their m...",CM
8,8,0.955556,1919,5180,Hot Tub Time Machine,2010,A malfunctioning time machine at a ski resort takes a man back to 1986 with his two friends and ...,CM,Hot Tub Time Machine,"March 26, 2010",A malfunctioning time machine at a ski resort takes a man back to 1986 with his two friends and ...,CM
9,9,0.955556,2011,5180,Hot Tub Time Machine,2010,A malfunctioning time machine at a ski resort takes a man back to 1986 with his two friends and ...,CM,Hot Tub Time Machine,"March 26, 2010",A malfunctioning time machine at a ski resort takes a man back to 1986 with his two friends and ...,CM


# Module: estimate_precision_recall
# Description: the below code helps you get an estimation of P/R on the candidate set.

In [7]:
import pandas as pd
from scipy.stats import norm
from numpy import sqrt

delta = .05
Z = norm.ppf(1 - (delta / 2))

def estimate_PR(labeled_pairs, reduced_cands, predicted_matches):
    '''
    labeled_pairs - a pandas dataframe with schema id1,id2,label
                    Note label needs to be Boolean

    reduced_cands - a pandas dataframe with schema id1,id2
    predicted_matches - a pandas dataframe with schema id1,id2
    
    return:
        ( (recall lower bound, recall upper bound), (precision lower bound, precision upper bound) )
    '''

    labeled_pairs.drop_duplicates(inplace=True)
    labeled_pairs.columns = ['id1', 'id2', 'label']
    reduced_cands.columns = ['id1', 'id2']
    reduced_cand_set = set(zip(reduced_cands.id1, reduced_cands.id2))
    predicted_matches = set(zip(predicted_matches.id1, predicted_matches.id2))
    
    # estimate the recall
    # number of positives in the labeled sample
    actual_pos = float(labeled_pairs.label.sum())
    # the maximum number of postives in the candidate set
    max_actual_pos = float(actual_pos + len(reduced_cand_set) - len(labeled_pairs))
    
    # true positives in the labeled sample
    true_pos = float(labeled_pairs.apply(lambda x : (x['id1'], x['id2']) in predicted_matches and x['label'], axis=1).sum())
    #estimated recall
    recall = float(true_pos / actual_pos)

    recall_error = Z * sqrt( ((recall * (1 - recall)) / (actual_pos)) * ((max_actual_pos - actual_pos) / (max_actual_pos - 1)) )


    # estimate Precision
    labeled_set  = set(zip(labeled_pairs.id1, labeled_pairs.id2))
    predicted_pos = float(len(labeled_set & predicted_matches))
    
    predicted_pos_in_reduced_cand_set = float(len(reduced_cand_set & predicted_matches))
    
    alpha =  predicted_pos_in_reduced_cand_set / len(predicted_matches)
    precision = alpha * (true_pos / predicted_pos)
    
    precision_error = alpha * Z * sqrt( ((precision * (1 - precision)) / predicted_pos) * (float((len(predicted_matches) - predicted_pos)) / (len(predicted_matches)  - 1)) )

    return ((recall - recall_error, recall + recall_error),
            (precision - precision_error, precision + precision_error))

# [NEED MODIFICATION: Modify this cell to point to the file location]

In [8]:
# read the labeled pairs file, i.e. the file with the labels
labeled_pairs = pd.read_csv('labeled.csv')
print(estimate_PR(labeled_pairs, dfc, dfp))

((1.0, 1.0), (1.0, 1.0))


# Alternatively if you run into issues running this script on your laptop you can use
# https://colab.research.google.com/notebooks/welcome.ipynb